In [1]:
import numpy as np
import networkx as nx
import scipy.sparse as sparse
from sklearn import preprocessing as pre

In [2]:
def locally_sensitive_hashing(m, d, w, sigma=1.0):
    # Compute random projection vector
    v = np.random.randn(d, 1) * sigma  # / np.random.randn(d, 1)

    # Compute random offset
    b = w * np.random.rand() * sigma

    # Compute hashes
    labels = np.floor((np.dot(m, v) + b) / w)

    # Compute label
    _, indices = np.unique(labels, return_inverse=True)

    return indices



def normalize_gram_matrix(x):
    k = np.reciprocal(np.sqrt(np.diag(x)))
    k = np.resize(k, (len(k), 1))
    return np.multiply(x, k.dot(k.T))


def compute_coloring(M, colors, log_primes):
    log_prime_colors = np.array([log_primes[i] for i in colors], dtype=np.float64)
    colors = colors + M.dot(log_prime_colors)

    # Round numbers to avoid numerical problems
    colors = np.round(colors, decimals=10)

    _, colors = np.unique(colors, return_inverse=True)

    return colors


In [3]:

import itertools as it
import numpy as np
import scipy.sparse.csr as csr
import scipy.sparse.lil as lil

def shortest_path_kernel(graph_db, hashed_attributes, *kwargs):
    use_labels = kwargs[0]

    num_vertices = 0
    for g in graph_db:
        num_vertices += g.number_of_nodes()

    offset = 0
    graph_indices = []
    colors_0 = np.zeros(num_vertices, dtype=np.int64)

    # Get labels (colors) from all graph instances
    offset = 0
    for g in graph_db:
        graph_indices.append((offset, offset + g.number_of_nodes() - 1))

        if use_labels == 1:
            for i, label in enumerate(nx.get_node_attributes(g,use_labels).values()):
                colors_0[i + offset] = label

        offset += g.number_of_nodes()
    _, colors_0 = np.unique(colors_0, return_inverse=True)

    colors_1 = hashed_attributes

    triple_indices = []
    triple_offset = 0
    triples = []

    # Solve APSP problem for every graphs in graph data base
    for i, g in enumerate(graph_db):
        # a = gt.adjacency(g)
        M = dict(nx.all_pairs_shortest_path_length(g))

        # index is a tuple giving index of first and last node for graph h
        index = graph_indices[i]

        if use_labels:
            l = colors_0[index[0]:index[1] + 1]
            h = colors_1[index[0]:index[1] + 1]
        else:
            h = colors_1[index[0]:index[1] + 1]
        d = len(M)

        # For each pair of vertices collect labels, hashed attributes, and shortest-path distance
        pairs = list(it.product(range(d), repeat=2))
        if use_labels:
            t = [hash((l[k], h[k], l[j], h[j], M[k][j])) for (k, j) in pairs if (k != j or ~np.isinf(M[k][j]))]
        else:
            t = [hash((h[k], h[j], M[k][j])) for (k, j) in pairs if (k != j or ~np.isinf(M[k][j]))]

        triples.extend(t)

        triple_indices.append((triple_offset, triple_offset + len(t) - 1))
        triple_offset += len(t)

    _, colors = np.unique(triples, return_inverse=True)
    m = np.amax(colors) + 1

    # Compute feature vectors
    feature_vectors = []
    for i, index in enumerate(triple_indices):
        feature_vectors.append(np.bincount(colors[index[0]:index[1] + 1], minlength=m))


    #if not compute_gram_matrix:
    return lil.lil_matrix(feature_vectors, dtype=np.float64) # each feature vector will be row
    # else:
    #     # Make feature vectors sparse
    #     gram_matrix = csr.csr_matrix(feature_vectors, dtype=np.float64) # each feature vector will be row
    #     # Compute gram matrix
    #     gram_matrix = gram_matrix.dot(gram_matrix.T)

    #     gram_matrix = gram_matrix.toarray()

    #     if normalize_gram_matrix:
    #         return normalize_gram_matrix(gram_matrix)
    #     else:
    #         return gram_matrix

In [16]:
import sys
#currentdir = os.path.dirname(os.path.realpath(__file__))
#parentdir = os.path.dirname(currentdir)
#sys.path.append(parentdir)
#parentdir = os.path.dirname(parentdir)
sys.path.append("../")
import MMDforGraphs as mg
from importlib import reload  
foo = reload(mg)
nr_nodes_1 = 30
nr_nodes_2 = 30
n = 3
m = 3


average_degree = 6


# bg1 = mg.BinomialGraphs(n, nr_nodes_1, average_degree, l = 'samelabels')
# bg1 = mg.BinomialGraphs(n, nr_nodes_1, average_degree, l = 'degreelabels')
bg1 = mg.BinomialGraphs(n, nr_nodes_1, average_degree, a = 'normattr', l = 'degreelabels' )
bg1.Generate()
bg2 = mg.BinomialGraphs(m, nr_nodes_2, average_degree+6, a = 'normattr', l = 'degreelabels', loc = 5)
bg2.Generate()

X = bg1.Gs + bg2.Gs

0
0
0
5
5
5


In [17]:

num_vertices = 0
for g in X:
    num_vertices += g.number_of_nodes()
n = len(X)

g = X[0]
tmp = nx.get_node_attributes(g,'attr')
dim_attributes = len(tmp[0])
print(f'dimension is {dim_attributes}')
colors_0 = np.zeros([num_vertices, dim_attributes])
offset = 0

gram_matrix = np.zeros([n, n])

# Get attributes from all graph instances
graph_indices = []
for g in X:
    for i, attr in enumerate(nx.get_node_attributes(g,'attr').values()):
        colors_0[i + offset] = attr

    graph_indices.append((offset, offset + g.number_of_nodes() - 1))
    offset += g.number_of_nodes()

# Normalize attributes: center to the mean and component wise scale to unit variance
if True:
    colors_0 = pre.scale(colors_0, axis=0)

colors_hashed = locally_sensitive_hashing(colors_0, dim_attributes, 0.1, sigma=1)

dimension is 1


In [27]:
colors_hashed

array([ 1,  3,  7,  4,  6,  1,  4,  3, 13, 11, 13, 11,  6,  9,  7,  5,  4,
       13, 11,  0,  9,  0,  7,  6,  9, 15,  5,  3,  5,  8,  8,  5, 12,  9,
       15,  7,  1, 11, 14, 11, 11, 13, 11,  7,  9, 10, 14,  5,  3,  5, 12,
        7,  9,  4,  6, 12, 10, 12,  5, 12, 11, 14,  8, 12, 11,  5,  8,  0,
       10, 12,  4,  8,  9, 12, 10,  9,  8,  6, 12, 11,  9,  9, 10,  9,  9,
       12,  3,  5,  2, 11, 18, 23, 25, 24, 21, 30, 16, 23, 23, 19, 21, 19,
       29, 28, 27, 24, 19, 26, 25, 20, 25, 23, 24, 25, 25, 30, 21, 24, 26,
       25, 18, 24, 26, 21, 27, 26, 29, 21, 21, 19, 30, 26, 21, 25, 24, 29,
       20, 19, 26, 19, 29, 26, 20, 23, 20, 29, 23, 17, 16, 28, 24, 16, 25,
       28, 23, 28, 24, 26, 28, 16, 23, 24, 21, 18, 24, 25, 24, 24, 24, 22,
       23, 23, 22, 28, 19, 27, 21, 24, 22, 27], dtype=int64)

In [26]:
import scipy as sp
import log_primes_list as log_pl
hashed_attributes = colors_hashed
# Create one empty feature vector for each graph
feature_vectors = []
for _ in X:
    feature_vectors.append(np.zeros(0, dtype=np.float64))

# Construct block diagonal matrix of all adjacency matrices
adjacency_matrices = []
for g in X:
    adjacency_matrices.append(np.array(nx.adjacency_matrix(g).todense()))
M = sp.sparse.block_diag(tuple(adjacency_matrices), dtype=np.float64, format="csr")
num_vertices = M.shape[0]
print(num_vertices)

# Load list of precalculated logarithms of prime numbers
log_primes = log_pl.log_primes[0:num_vertices]

# Color vector representing labels
colors_0 = np.zeros(num_vertices, dtype=np.float64)
# Color vector representing hashed attributes
colors_1 = hashed_attributes

# Get labels (colors) from all graph instances
offset = 0
graph_indices = []


for g in graph_db:
    if False:
        for i, label in enumerate(nx.get_node_attributes(g,label_name).values()):
            colors_0[i + offset] = label

    graph_indices.append((offset, offset + g.num_vertices() - 1))
    offset += g.num_vertices()

# Map labels to [0, number_of_colors)
if False:
    _, colors_0 = np.unique(colors_0, return_inverse=True)

for it in range(0, iterations + 1):

    if False:
        # Map colors into a single color vector
        colors_all = np.array([colors_0, colors_1])
        colors_all = [hash(tuple(row)) for row in colors_all.T]
        _, colors_all = np.unique(colors_all, return_inverse=True)
        max_all = int(np.amax(colors_all) + 1)
        # max_all = int(np.amax(colors_0) + 1)

        feature_vectors = [
            np.concatenate((feature_vectors[i], np.bincount(colors_all[index[0]:index[1] + 1], minlength=max_all)))
            for i, index in enumerate(graph_indices)]

        # Avoid coloring computation in last iteration
        if it < iterations:
            colors_0 = compute_coloring(M, colors_0, log_primes[0:len(colors_0)])
            colors_1 = compute_coloring(M, colors_1, log_primes[0:len(colors_1)])
    else:
        max_1 = int(np.amax(colors_1) + 1)

        feature_vectors = [
            np.concatenate((feature_vectors[i], np.bincount(colors_1[index[0]:index[1] + 1], minlength=max_1))) for
            i, index in enumerate(graph_indices)]

        # Avoid coloring computation in last iteration
        if it < iterations:
            colors_1 = compute_coloring(M, colors_1, log_primes[0:len(colors_1)])

# if not compute_gram_matrix:
#     return lil.lil_matrix(feature_vectors, dtype=np.float64)
# else:
#     # Make feature vectors sparse
#     gram_matrix = csr.csr_matrix(feature_vectors, dtype=np.float64)
#     # Compute gram matrix
#     gram_matrix = gram_matrix.dot(gram_matrix.T)

#     gram_matrix = gram_matrix.toarray()

#     if normalize_gram_matrix:
#         return aux.normalize_gram_matrix(gram_matrix)
#     else:
#         return gram_matrix

180
